In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 111.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.0 MB/s eta 0:00:00


In [2]:
!pip install transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.5 MB/s eta 0:00:00


In [3]:
!pip install xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 7.7 MB/s eta 0:00:00


In [4]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.4 MB/s eta 0:00:00


In [5]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.2 MB/s eta 0:00:00


In [6]:
!pip install huggingface_hub

In [7]:
from datasets import load_dataset #HF библиотека по работе с датасэтами
from transformers import AutoTokenizer #автоматическое определение токенайзера предобученной модели
from transformers import DataCollatorWithPadding #набивка токенов до единой размерности
from transformers import TFAutoModelForSequenceClassification #головная часть модели для решения GLUE задачи
from tensorflow.keras.losses import SparseCategoricalCrossentropy #loss function
from tensorflow.keras.optimizers.schedules import PolynomialDecay #уменьшает learning rate по ходу обучения
from tensorflow.keras.optimizers import Adam #optimizer
import tensorflow as tf
import evaluate #оценка результатов
import numpy as np
from transformers import PushToHubCallback

In [8]:
from huggingface_hub import notebook_login

notebook_login()

In [13]:
raw_datasets = load_dataset('SIA86/TechnicalSupportCalls') #с помощью библиотеки datasets загружаем датасет с сайта HF
checkpoint = "microsoft/mdeberta-v3-base" #выбираем модель
tokenizer = AutoTokenizer.from_pretrained(checkpoint) #загружаем токенайзер из предобученной модели
#tokenizer.add_special_tokens({'pad_token': '[PAD]'}) #add for gpt3

def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True) #токенизируем скаченный датасет
tokenized_datasets = tokenized_datasets.map(lambda examples: {"labels": examples["label"]}, batched=True)
#tokenized_datasets = tokenized_datasets.rename_column("label", "labels") #переименование колонки label



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/921 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/231 [00:00<?, ? examples/s]

Map:   0%|          | 0/921 [00:00<?, ? examples/s]

Map:   0%|          | 0/231 [00:00<?, ? examples/s]

In [14]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 921
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 231
    })
})

In [15]:
tokenized_datasets['train'][15]['input_ids']

[1,
 60276,
 1109,
 173839,
 20154,
 680,
 654,
 636,
 260,
 265,
 382,
 260,
 265,
 20293,
 1131,
 70466,
 553,
 8201,
 21496,
 91619,
 275,
 355,
 73940,
 260,
 225783,
 272,
 2]

In [ ]:
from keras.api._v2.keras import callbacks
data_collator = DataCollatorWithPadding(tokenizer=tokenizer) #подгружаем дата коллатор

tf_train_dataset = tokenized_datasets["train"].to_tf_dataset( #переводим датасэт в tf.dataset формат
    columns=["attention_mask", "input_ids", "token_type_ids"], #оставляем только нужные колонки
    label_cols=["labels"], #передаем колонку с таргетами
    shuffle=False,
    collate_fn=data_collator, #выравнивание токенов по длинне
    batch_size=8,
)


tf_test_dataset = tokenized_datasets["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

callback = PushToHubCallback(
    "robera_larde_text_class", save_strategy="epoch", tokenizer=tokenizer)


num_epochs = 10

num_train_steps = len(tf_train_dataset) * num_epochs #определяем длинну шагов (колическо элементов в датасете/на бэтчсайз * эпохи)

lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=5e-8, decay_steps=num_train_steps
)
opt = Adam(learning_rate=lr_scheduler) #настраиваем leraning rate таким образом чтобы он уменьшался по ходу обучения

model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=38) #создаем головную часть модели указываем количество лэйблов
#model = TFAutoModelForTokenClassification.from_pretrained(checkpoint, num_labels=5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) #настраиваем loss func на прием logits
model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])
model.fit(tf_train_dataset, validation_data=tf_test_dataset, epochs=num_epochs, callbacks=[callback])